In [18]:
import pandas as pd
import keras
from keras import layers
import keras_tuner as kt

In [19]:
df = pd.read_csv("/content/Real_Combine.csv")

In [20]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [21]:
df.isna().sum()

,0
T,0
TM,0
Tm,0
SLP,0
H,0
VV,0
V,0
VM,0
PM 2.5,1


In [22]:
df = df.dropna()
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [23]:
def build_model(hp, input_shape):
  model = keras.Sequential()
  model.add(keras.Input(shape=(input_shape,)))

  for i in range(hp.Int('num_layers',2,8)):
    model.add(layers.Dense(units=hp.Int('units_'+str(i), min_value=32, max_value=512, step=32), activation='relu', kernel_initializer='he_uniform'))
    model.add(layers.Dense(1, activation='linear'))

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error']
    )
  return model


In [24]:
n_features = X.shape[1]
tuner = kt.RandomSearch(
    lambda hp: build_model(hp, input_shape=n_features),
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='/content/project/',
    project_name="Air_Quality_Index"
)

In [25]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}


In [26]:
from sklearn.model_selection import train_test_split
X_trian, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [27]:
tuner.search(X_trian, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 19s]
val_mean_absolute_error: 57.93571853637695

Best val_mean_absolute_error So Far: 57.93571853637695
Total elapsed time: 00h 01m 41s


In [28]:
tuner.results_summary()

Results summary
Results in /content/project/Air_Quality_Index
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 4 summary
Hyperparameters:
num_layers: 5
units_0: 256
learning_rate: 0.001
units_1: 128
units_2: 32
units_3: 224
units_4: 416
units_5: 480
units_6: 224
Score: 57.93571853637695

Trial 1 summary
Hyperparameters:
num_layers: 7
units_0: 224
learning_rate: 0.001
units_1: 128
units_2: 192
units_3: 288
units_4: 96
units_5: 288
units_6: 32
Score: 59.14627202351888

Trial 3 summary
Hyperparameters:
num_layers: 7
units_0: 192
learning_rate: 0.01
units_1: 256
units_2: 64
units_3: 352
units_4: 32
units_5: 512
units_6: 128
Score: 63.185526529947914

Trial 2 summary
Hyperparameters:
num_layers: 5
units_0: 384
learning_rate: 0.01
units_1: 416
units_2: 64
units_3: 512
units_4: 352
units_5: 512
units_6: 288
Score: 66.62277730305989

Trial 0 summary
Hyperparameters:
num_layers: 6
units_0: 160
learning_rate: 0.001
units_1: 480
units_2: 32
units_3: 32
unit

In [31]:
best_model = tuner.get_best_models(num_models=1)[0]

history = best_model.fit(X_trian, y_train, epochs=50, validation_data = (X_test, y_test))

test_loss, test_mae = best_model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 91.9657 - mean_absolute_error: 91.9657 - val_loss: 70.3787 - val_mean_absolute_error: 70.3787
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 70.9487 - mean_absolute_error: 70.9487 - val_loss: 71.6940 - val_mean_absolute_error: 71.6940
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 75.8617 - mean_absolute_error: 75.8617 - val_loss: 67.2245 - val_mean_absolute_error: 67.2245
Epoch 4/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 68.6511 - mean_absolute_error: 68.6511 - val_loss: 67.3089 - val_mean_absolute_error: 67.3089
Epoch 5/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 72.9854 - mean_absolute_error: 72.9854 - val_loss: 68.8958 - val_mean_absolute_error: 68.8958
Epoch 6/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 72.7300 - mean_absolute_error: 72.7300 - val_loss: 68.5626 - val_mean_absolute_error: 68.5626
Epoch 7/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 75.3405 - mean_absolute_error: 75.3405 -